# Imports

In [1]:
!pip install tables
!pip install torchvision

  Using cached tables-3.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached numexpr-2.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (381 kB)
  Using cached blosc2-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached torchvision-0.15.2-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_cupti_cu11-11.7.101-py3-none-manylinux1_x86_64.whl (11.8 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-p

In [21]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=12)
import pandas as pd
pd.set_option('display.precision', 12)
from sklearn import *
import sys
import subprocess
from datetime import datetime, timedelta
import netCDF4
import time
from functools import partial
import os
import requests


# Ensure that working directory is forecast_rodeo
if os.path.basename(os.getcwd()) == "experiments":
    # Navigate to forecast_rodeo
    os.chdir(os.path.join("..",".."))
if os.path.basename(os.getcwd()) != "forecast_rodeo_upgrade":
    raise Exception("You must be in the forecast_rodeo_upgrade folder")

# Adds 'experiments' folder to path to load experiments_util
sys.path.insert(0, 'src/experiments')
# Load general utility functions
from experiments_util import *

# Experiment name
experiment = "neural_network"

#####

import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#import shap

# Choose target
#
gt_id = "contest_tmp2m" # "contest_precip" or "contest_tmp2m"
target_horizon = "34w" # "34w" or "56w"

##CAUTION ! It seeems that for both contest_tmp2m 34w and contest precip 56w all the CASM GPP RECO have been added correctly. But apparently for contest_tmp2m 56w and 

# Create list of official contest submission dates in YYYYMMDD format
#
submission_dates = [datetime(y,4,18)+timedelta(14*i) for y in range(2011,2018) for i in range(26)]
submission_dates = ['{}{:02d}{:02d}'.format(date.year, date.month, date.day) for date in submission_dates]
submission_dates = [datetime.strptime(str(d), "%Y%m%d") for d in submission_dates]
submission_dates = pd.Series(submission_dates)

# Create list of target dates corresponding to submission dates in YYYYMMDD format
#
target_dates = pd.Series([get_target_date('{}{:02d}{:02d}'.format(date.year, date.month, date.day), target_horizon) for date in submission_dates])

# Find all unique target day-month combinations
target_day_months = pd.DataFrame({'month' : target_dates.dt.month, 
                                  'day': target_dates.dt.day}).drop_duplicates()

#####

from experiments_util import *
# Load functionality for fitting and predicting
from fit_and_predict import *
# Load functionality for evaluation
from skill import *
# Load functionality for stepwise regression
from stepwise_util import *

hindcast_features = False if len(sys.argv) < 6 else (sys.argv[5] == "True")
print(hindcast_features)

# Identify measurement variable name
measurement_variable = get_measurement_variable(gt_id) # 'tmp2m' or 'precip'

# column names for gt_col, clim_col and anom_col 
gt_col = measurement_variable
clim_col = measurement_variable+"_clim"
anom_col = get_measurement_variable(gt_id)+"_anom" # 'tmp2m_anom' or 'precip_anom'

# anom_inv_std_col: column name of inverse standard deviation of anomalies for each start_date
anom_inv_std_col = anom_col+"_inv_std"

#
# Default regression parameter values
#
# anom_scale_col: multiply anom_col by this amount prior to prediction
# (e.g., 'ones' or anom_inv_std_col)
anom_scale_col = 'ones'
# pred_anom_scale_col: multiply predicted anomalies by this amount
# (e.g., 'ones' or anom_inv_std_col)
pred_anom_scale_col = 'ones'
# choose first year to use in training set # (before 1979 = bad)
first_train_year = 1979 if gt_id == 'contest_precip' else 1979
# columns to group by when fitting regressions (a separate regression
# is fit for each group); use ['ones'] to fit a single regression to all points
group_by_cols = ['lat', 'lon']
# base_col: column which should be subtracted from gt_col prior to prediction
# (e.g., this might be clim_col or a baseline predictor like NMME)
base_col = 'zeros'
#
# Default stepwise parameter values
#
# Define candidate predictors
initial_candidate_x_cols = default_stepwise_candidate_predictors(gt_id, target_horizon, hindcast=hindcast_features)
# Copy the list of candidates for later modification
candidate_x_cols = initial_candidate_x_cols[:]
# Skill threshold for what counts as a similar year
similar_years_threshold = 0.1
# Tolerance for convergence: if improvement is less than tolerance, terminate.
tolerance = 0.01
# Whether to use margin days (days around the target date)
use_margin = False


date_data = pd.read_hdf('results/regression/shared/' + gt_id + '_' + target_horizon + '/date_data-'+ gt_id + '_' + target_horizon + '.h5')
lat_lon_date_data = pd.read_hdf('results/regression/shared/' + gt_id + '_' + target_horizon + '/lat_lon_date_data-'+ gt_id + '_' + target_horizon + '.h5')
print('lat lon colums', lat_lon_date_data.columns)

if target_horizon == "34w":
    relevant_cols = set(candidate_x_cols+[base_col,clim_col,anom_col,'start_date','lat','lon','precip','GPP_shift30','RECO_shift30','CASM_soil_moisture_shift29']+group_by_cols)
if target_horizon == "56w":
    relevant_cols = set(candidate_x_cols+[base_col,clim_col,anom_col,'start_date','lat','lon','precip','GPP_shift44','RECO_shift44','CASM_soil_moisture_shift43']+group_by_cols)
print('relevant_cols', relevant_cols)
data = lat_lon_date_data.loc[lat_lon_date_data.start_date.dt.year >= first_train_year,
                             lat_lon_date_data.columns.isin(relevant_cols)]
data = pd.merge(data, date_data.loc[date_data.start_date.dt.year >= first_train_year,
                                    date_data.columns.isin(relevant_cols)],
                on="start_date", how="left")
print('data colums', data.columns)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
False
lat lon colums Index(['lat', 'lon', 'key_2', 'key_3', 'start_date', 'rhum_shift30',
       'pres_shift30', 'GPP_shift30', 'RECO_shift30', 'Unnamed: 0_shift29',
       'CASM_soil_moisture_shift29', 'rhum_shift60', 'pres_shift60',
       'GPP_shift60', 'RECO_shift60', 'Unnamed: 0_shift58',
       'CASM_soil_moisture_shift58', 'rhum_shift365', 'pres_shift365',
       'GPP_shift365', 'RECO_shift365', 'Unnamed: 0_shift365',
       'CASM_soil_moisture_shift365', 'ccsm3', 'nasa', 'nmme_mean',
       'nmme_wo_ccsm3_nasa', 'ccsm3_0', 'nasa_0', 'nmme0_mean',
       'nmme0_wo_ccsm3_nasa', 'ccsm3_shift15', 'nasa_shift15',
       'nmme_mean_shift15', 'nmme_wo_ccsm3_nasa_shift15', 'ccsm3_0_shift15',
       'nasa_0_shift15', 'nmme0_mean_shift15', 'nmme0_wo_ccsm3_nasa_shift15',
       'tmp2m', 'tmp2m_sqd', 'tmp2m_std', 'tmp2m_clim', 'tmp2m_anom',
       'precip_shift29', 'precip_shift29_clim', 'precip_shift29

# Process of the data

## Restriction of the data through time

In [22]:
# Restriction over the dates
condition00 = (data['start_date'] >= '1979-01-01')
condition01 = (data['start_date'] <= '2018-05-23')
data = data.drop(data[~condition00].index)
data = data.drop(data[~condition01].index)
print(data['start_date'])

# delete the duplicates to have a list of all the different existing dates
extract_time = data['start_date'].drop_duplicates().sort_values() #
print(extract_time.shape)
             
             

0          1979-01-01
1          1979-01-02
2          1979-01-03
3          1979-01-04
4          1979-01-05
              ...    
44285680   2018-05-07
44285681   2018-05-10
44285682   2018-05-13
44285683   2018-05-16
44285684   2018-05-22
Name: start_date, Length: 38264313, dtype: datetime64[ns]
(14388,)


## Restriction of the data through latitude and longitude

In [23]:
# Keeps only the latitude and longitudes values who are integers
condition5 = data['lat'].astype(int) == data['lat']
data = data.drop(data[~condition5].index)
condition6 = data['lon'].astype(int) == data['lon']
data = data.drop(data[~condition6].index)

#restriction to the smallest rectangle possible
condition1 = (data['lat'] < 27.0) | (data['lat'] > 49.0)
condition2 = (data['lon'] < 236.0) | (data['lon'] > 266.0)
data = data.drop(data[condition1].index)
data = data.drop(data[condition2].index)

#creates vectors containing all the possible values of latitude and longitude, dropping duplicates
latitude_values = data['lat'].drop_duplicates().sort_values()
longitude_values = data['lon'].drop_duplicates().sort_values()
latitude_values = np.array(latitude_values)
longitude_values = np.array(longitude_values)
print(latitude_values)
print(longitude_values)

[27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0 37.0 38.0 39.0 40.0
 41.0 42.0 43.0 44.0 45.0 46.0 47.0 48.0 49.0]
[236.0 237.0 238.0 239.0 240.0 241.0 242.0 243.0 244.0 245.0 246.0 247.0
 248.0 249.0 250.0 251.0 252.0 253.0 254.0 255.0 256.0 257.0 258.0 259.0
 260.0 261.0 262.0 263.0 264.0 265.0 266.0]


## Creation and save of the Mask
Can be done only one time, because it is a long step. A mask is already saved in results/matrix.

In [11]:
#Determine minimum and maximum latitude and longitude values
min_latitude = np.min(latitude_values)
max_latitude = np.max(latitude_values)
min_longitude = np.min(longitude_values)
max_longitude = np.max(longitude_values)

#Create meshgrid of latitude and longitude values
grid_latitude, grid_longitude = np.meshgrid(np.arange(min_latitude, max_latitude + 1), np.arange(min_longitude, max_longitude + 1))
print('g',grid_latitude.shape)
print('gk',grid_latitude.shape)
# Check existence of grid points in the dataframe
mask = np.zeros_like(grid_latitude, dtype=bool)
for i in range(len(latitude_values)):
    print(i)
    lat_idx = np.where(grid_latitude == latitude_values[i])
    for j in range(len(longitude_values)):
        lon_idx = np.where(grid_longitude == longitude_values[j])
        exists_in_data = (data['lat'] == latitude_values[i]) & (data['lon'] == longitude_values[j])
        if exists_in_data.any():
            mask[j, i] = True

# Create the final mask
final_mask = np.logical_not(mask)
print(final_mask)

final_mask = th.tensor(final_mask)
final_mask = final_mask.permute(1,0)
print(final_mask.shape)
# Name of cache directory for storing non-submission-date specific
# intermediate files
cache_dir = os.path.join('results', 'matrix')
# e.g., cache_dir = 'results/regression/shared/contest_precip_34w'

# if cache_dir doesn't exist, create it
if not os.path.isdir(cache_dir):
    os.makedirs(cache_dir)

# Filenames for data file to be stored in cache_dir
data_file = os.path.join(
    cache_dir, "mask_tensor_permuted_TEST.tensor")

print("Saving multiarrays features to " + data_file)
th.save(final_mask, data_file)

print("Finished generating data matrix.")

g (31, 23)
gk (31, 23)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True False False False False False False  True False False  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
  False False False False False False False False False False  True]
 [ True  True  True  True  True  True  True  True  True  True False False
  False False False False False False False False False False False]
 [ True  True  True  True  True  True  True  True  True False False False
  False False False False False False False False False False False]
 [ True  True  True  True  True  True  True  True False False False False
  False False False False False False False False False False False]
 [ True  True  True  True  True  True  True  True False False False False
  False False False False False False False False False False False]
 [ True  True  True  True  True  True  True False False Fals

## Copy of the data in another dataframe

In [24]:
#copy of the data and counting the NaN values

filtered_df = data
#only keeps th integer values, for simplification
condition5 = filtered_df['lat'].astype(int) == filtered_df['lat']
filtered_df = filtered_df.drop(filtered_df[~condition5].index)
condition6 = filtered_df['lon'].astype(int) == filtered_df['lon']
filtered_df = filtered_df.drop(filtered_df[~condition6].index)

print('columns',filtered_df.columns)
# Count NaN values by column
print("shape",filtered_df.shape)
nan_counts = filtered_df.isna().sum()
print('NaN counts', nan_counts)

columns Index(['lat', 'lon', 'start_date', 'rhum_shift30', 'pres_shift30',
       'GPP_shift30', 'RECO_shift30', 'CASM_soil_moisture_shift29',
       'nmme_wo_ccsm3_nasa', 'nmme0_wo_ccsm3_nasa', 'tmp2m_clim', 'tmp2m_anom',
       'tmp2m_shift29', 'tmp2m_shift29_anom', 'tmp2m_shift58',
       'tmp2m_shift58_anom', 'mei_shift45', 'phase_shift17',
       'sst_2010_1_shift30', 'sst_2010_2_shift30', 'sst_2010_3_shift30',
       'icec_2010_1_shift30', 'icec_2010_2_shift30', 'icec_2010_3_shift30',
       'wind_hgt_10_2010_1_shift30', 'wind_hgt_10_2010_2_shift30'],
      dtype='object')
shape (7485181, 26)
NaN counts lat                                 0
lon                                 0
start_date                          0
rhum_shift30                    89749
pres_shift30                    89749
GPP_shift30                   7357469
RECO_shift30                  7357469
CASM_soil_moisture_shift29    7485181
nmme_wo_ccsm3_nasa             653093
nmme0_wo_ccsm3_nasa            653093
tmp

## Soil Moisture, GPP, RECO Preprocess

In [25]:
### Soil Moisture

#creation of a new dataframe based on what has been created in lat_lon_date_data thanks to the "create_data_matrices" program.
df_CASM=pd.DataFrame()
df_CASM['lat']=lat_lon_date_data['lat']
df_CASM['lon']=lat_lon_date_data['lon']
if target_horizon == "34w":
    df_CASM['CASM']=lat_lon_date_data['CASM_soil_moisture_shift29']
if target_horizon == "56w":
    df_CASM['CASM']=lat_lon_date_data['CASM_soil_moisture_shift43']
df_CASM['start_date']=lat_lon_date_data['start_date']

df_CASM_san=df_CASM.dropna(subset=['CASM'])

#fill the NaN values with values around
df_CASM_san['lat'] = df_CASM_san['lat'].fillna(method='ffill').fillna(method='bfill')
df_CASM_san['lon'] = df_CASM_san['lon'].fillna(method='ffill').fillna(method='bfill')

#Rounds the values to integer
df_CASM_san['lat']=df_CASM_san['lat'].round()
df_CASM_san['lon']=df_CASM_san['lon'].round()

#onnly keeps innteger values andn drop dupliactes
condition5 = df_CASM_san['lat'].astype(int) == df_CASM_san['lat']
df_CASM_san = df_CASM_san.drop(df_CASM_san[~condition5].index)
condition6 = df_CASM_san['lon'].astype(int) == df_CASM_san['lon']
df_CASM_san = df_CASM_san.drop(df_CASM_san[~condition6].index)
latitude_values = df_CASM_san['lat'].drop_duplicates().sort_values()
longitude_values = df_CASM_san['lon'].drop_duplicates().sort_values()

### Same process for GPP

df_GPP=pd.DataFrame()
df_GPP['lat']=lat_lon_date_data['lat']
df_GPP['lon']=lat_lon_date_data['lon']
if target_horizon == "34w":
    df_GPP['GPP']=lat_lon_date_data['GPP_shift30']
if target_horizon == "56w":
    df_GPP['GPP']=lat_lon_date_data['GPP_shift44']
df_GPP['start_date']=lat_lon_date_data['start_date']

df_GPP_san=df_GPP.dropna(subset=['GPP'])

df_GPP_san['lat'] = df_GPP_san['lat'].fillna(method='ffill').fillna(method='bfill')
df_GPP_san['lon'] = df_GPP_san['lon'].fillna(method='ffill').fillna(method='bfill')

df_GPP_san['lat']=df_GPP_san['lat'].round()
df_GPP_san['lon']=df_GPP_san['lon'].round()

condition5 = df_GPP_san['lat'].astype(int) == df_GPP_san['lat']
df_GPP_san = df_GPP_san.drop(df_GPP_san[~condition5].index)
condition6 = df_GPP_san['lon'].astype(int) == df_GPP_san['lon']
df_GPP_san = df_GPP_san.drop(df_GPP_san[~condition6].index)
latitude_values = df_GPP_san['lat'].drop_duplicates().sort_values()
longitude_values = df_GPP_san['lon'].drop_duplicates().sort_values()


### Same process for RECO
df_RECO=pd.DataFrame()
df_RECO['lat']=lat_lon_date_data['lat']
df_RECO['lon']=lat_lon_date_data['lon']
if target_horizon == "34w":
    df_RECO['RECO']=lat_lon_date_data['RECO_shift30']
if target_horizon == "56w":
    df_RECO['RECO']=lat_lon_date_data['RECO_shift44']
df_RECO['start_date']=lat_lon_date_data['start_date']

df_RECO_san=df_RECO.dropna(subset=['RECO'])

df_RECO_san['lat'] = df_RECO_san['lat'].fillna(method='ffill').fillna(method='bfill')
df_RECO_san['lon'] = df_RECO_san['lon'].fillna(method='ffill').fillna(method='bfill')

df_RECO_san['lat']=df_RECO_san['lat'].round()
df_RECO_san['lon']=df_RECO_san['lon'].round()

condition5 = df_RECO_san['lat'].astype(int) == df_RECO_san['lat']
df_RECO_san = df_RECO_san.drop(df_GPP_san[~condition5].index)
condition6 = df_RECO_san['lon'].astype(int) == df_RECO_san['lon']
df_RECO_san = df_RECO_san.drop(df_RECO_san[~condition6].index)
latitude_values = df_RECO_san['lat'].drop_duplicates().sort_values()
longitude_values = df_RECO_san['lon'].drop_duplicates().sort_values()

## count NaN values in these dataframes 
print(df_CASM_san.shape)
nan_counts = df_CASM_san.isna().sum()
print(nan_counts)

print(df_GPP_san.shape)
nan_counts = df_GPP_san.isna().sum()
print(nan_counts)

print(df_RECO_san.shape)
nan_counts = df_RECO_san.isna().sum()
print(nan_counts)

/tmp/ipykernel_453/4285619449.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CASM_san['lat'] = df_CASM_san['lat'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_453/4285619449.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CASM_san['lon'] = df_CASM_san['lon'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_453/4285619449.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

(14348406, 4)
lat           0
lon           0
CASM          0
start_date    0
dtype: int64
(2363760, 4)
lat           0
lon           0
GPP           0
start_date    0
dtype: int64
(2363760, 4)
lat           0
lon           0
RECO          0
start_date    0
dtype: int64


In [26]:
new_df_CASM = df_CASM_san.groupby(['lat', 'lon', 'start_date'], as_index=False)['CASM'].mean()
merged_df = filtered_df.merge(new_df_CASM[['lat', 'lon', 'start_date', 'CASM']], on=['lat', 'lon', 'start_date'], how='left')
new_df_GPP = df_GPP_san.groupby(['lat', 'lon', 'start_date'], as_index=False)['GPP'].mean()
merged_df = merged_df.merge(new_df_GPP[['lat', 'lon', 'start_date', 'GPP']], on=['lat', 'lon', 'start_date'], how='left')
new_df_RECO = df_RECO_san.groupby(['lat', 'lon', 'start_date'], as_index=False)['RECO'].mean()
merged_df = merged_df.merge(new_df_RECO[['lat', 'lon', 'start_date', 'RECO']], on=['lat', 'lon', 'start_date'], how='left')
filtered_df = merged_df #.drop(['CASM_soil_moisture_shift44 '])

print(filtered_df.shape)
nan_counts = filtered_df.isna().sum()
print(nan_counts)

(7485181, 29)
lat                                 0
lon                                 0
start_date                          0
rhum_shift30                    89749
pres_shift30                    89749
GPP_shift30                   7357469
RECO_shift30                  7357469
CASM_soil_moisture_shift29    7485181
nmme_wo_ccsm3_nasa             653093
nmme0_wo_ccsm3_nasa            653093
tmp2m_clim                      94889
tmp2m_anom                      94889
tmp2m_shift29                  104655
tmp2m_shift29_anom             109795
tmp2m_shift58                  119561
tmp2m_shift58_anom             124701
mei_shift45                         0
phase_shift17                    8738
sst_2010_1_shift30             516056
sst_2010_2_shift30             516056
sst_2010_3_shift30             516056
icec_2010_1_shift30            516056
icec_2010_2_shift30            516056
icec_2010_3_shift30            516056
wind_hgt_10_2010_1_shift30          0
wind_hgt_10_2010_2_shift30          

## Transform temporal data from Timestamp to Integers

In [27]:
# necessary step, timestamp type of date doesn't fit i Pytorch tensors so we have to convert it into integers.
filtered_df['start_date'] = filtered_df['start_date'].view('int64').astype(int) #
filtered_df = filtered_df.reset_index(drop=True) #line 0 starts now ayt index 0 after modifications

## Creation of the Elevation Tensor

In [28]:
t=time.time()
#loads the elevation dataset
dataset = xr.open_dataset('new_features/elevationdata.nc')
latitudes = dataset['lat'].values
latitudes=np.repeat(latitudes,dataset.sizes['lon']) 
longitudes = dataset['lon'].values
elevations=dataset['elevation'].values

gt = pd.DataFrame({
    'lat': latitudes
})
longitudes_repeated = np.tile(longitudes, dataset.sizes['lat']) #585
df_lon = pd.DataFrame({'lon': longitudes_repeated})
elev=[]
for lat in elevations:
    for lon in lat:
        elev.append(lon)
df_elev=pd.DataFrame({'elevation':elev})
gt=pd.concat([gt,df_lon,df_elev],axis=1)
if isinstance(gt.index, pd.MultiIndex):
    gt.reset_index(inplace=True)

lat_restriction_left = 27 #restriction of data around the US (latitude North)
lat_restriction_right = 49 #restriction of data around the US (latitude North)
lon_restriction_left = -124 #restriction of data around the US (longitude West)
lon_restriction_right = -94 #restriction of data aroud the US (longitude West)

#only keep the values inside
gt = gt[gt['lat'].between(lat_restriction_left, lat_restriction_right)] 
gt = gt[gt['lon'].between(lon_restriction_left, lon_restriction_right)]

gt['lon'] = np.where(gt['lon']< 0, gt['lon'] + 360, gt['lon'])

gt['lat_rounded']=gt['lat'].round()
gt['lon_rounded']=gt['lon'].round()

gt_new = gt.groupby(['lat_rounded', 'lon_rounded'], as_index=False)['elevation'].mean()

elevation_df=gt_new.rename(columns={'lat_rounded':'lat','lon_rounded':'lon'})
print(elevation_df)
print('Elevation loading time=',time.time()-t)

tic()
extract_latitudes = elevation_df['lat'].drop_duplicates().sort_values()
extract_longitudes = elevation_df['lon'].drop_duplicates().sort_values()
extract_latitudes =np.array(extract_latitudes).tolist()
extract_longitudes =np.array(extract_longitudes).tolist()
extract_features = np.array(elevation_df.columns).tolist()
# Create the coordinate map
coordinate_map = {}
for lat_idx, lat in enumerate(extract_latitudes):
    for lon_idx, lon in enumerate(extract_longitudes):
        for features_idx, feature in enumerate(extract_features):
            coordinate_map[(lat, lon, feature)] = (lat_idx, lon_idx, features_idx)

# Create an empty tensor with the desired dimensions
tensor_elevation = th.empty((len(extract_latitudes), len(extract_longitudes), len(extract_features)))

# Iterate over the dataframe rows and fill the tensor using the coordinate map
for columns in elevation_df.columns:
    print(columns)
    tic()
    for _, row in elevation_df.iterrows():
        lat, lon, feature = row['lat'], row['lon'], row[columns]
        tensor_elevation[coordinate_map[(lat, lon, columns)]] = feature
    toc()
# Print the filled tensor
print(tensor_elevation.shape)

cache_dir = os.path.join('results', 'matrix')
# e.g., cache_dir = 'results/regression/shared/contest_precip_34w'

# if cache_dir doesn't exist, create it
if not os.path.isdir(cache_dir):
    os.makedirs(cache_dir)

# Filenames for data file to be stored in cache_dir
data_file = os.path.join(
    cache_dir, "elevation_TEST.tensor")

print("Saving multiarrays features to " + data_file)
th.save(tensor_elevation, data_file)

print("Finished generating data matrix.")



      lat    lon         elevation
0    27.0  236.0    0.000000000000
1    27.0  237.0    0.000000000000
2    27.0  238.0    0.000000000000
3    27.0  239.0    0.000000000000
4    27.0  240.0    0.000000000000
..    ...    ...               ...
708  49.0  262.0  392.001111111111
709  49.0  263.0  262.337500000000
710  49.0  264.0  328.661805555556
711  49.0  265.0  330.006111111111
712  49.0  266.0  163.785277777778

[713 rows x 3 columns]
Elevation loading time= 3.422720193862915
lat
Elapsed time: 0.049385 seconds.

lon
Elapsed time: 0.037028 seconds.

elevation
Elapsed time: 0.037712 seconds.

torch.Size([23, 31, 3])
Saving multiarrays features to results/matrix/elevation_TEST.tensor
Finished generating data matrix.


## Creation of ElNino Tensor

In [31]:
import requests
elnino=['1','34','4']

elNino={'elnino1': None, 'elnino34': None, 'elnino4': None}


for elnin in elnino:
    url = "https://psl.noaa.gov/data/correlation/nina{}.data".format(elnin) #El Nino 1

    response = requests.get(url)
    data = response.text

    # Split the string into lines
    lines = data.split('\n')

    # Remove the first line
    lines = lines[32:]

    # Remove the last two lines
    lines = lines[:-8]

    # Join the remaining lines back into a string
    modified_data = '\n'.join(lines)


    # Assuming the modified_data variable contains the modified string data
    # Split the modified data into lines
    lines = modified_data.split('\n')
    # Initialize empty lists for columns
    columns = [[] for _ in range(13)]

    # Iterate over each line and extract values for each column
    for line in lines:
        values = line.split(' ')
        filtered_values = [value for value in values if value != '']

        for i in range(0, 13):  # Extract values for columns 2 to 13
            columns[i-1].append(filtered_values[i])

    # Create a dictionary of columns
    data_dict = {f"Column {i+2}": col for i, col in enumerate(columns)}
    # Create a DataFrame
    df = pd.DataFrame(data_dict)

    df['Column 14'] = df['Column 14'].str.strip().astype(float)

    # Set the index based on the first column (year)
    df['Column 14'] = pd.to_datetime(df['Column 14'], format='%Y')

    # Convert the index to datetime
    df.set_index('Column 14', inplace=True)

    for i in range(3,13):
        df[f'Column {i}'] = df[f'Column {i}'].replace('', np.nan)
        df[f'Column {i}'].str.strip().astype(float)

    stacked_df = df.stack()

    # Convertir la série résultante en un nouveau dataframe avec une seule colonne
    df1 = pd.DataFrame(stacked_df,columns=['El Nino'])
    df1.reset_index(drop=True,inplace=True)

    # Générer une séquence de dates mensuelles
    start_date = '1979-01'
    end_date = '2023-12'
    idx = pd.date_range(start=start_date, end=end_date, freq='MS')

    # Assiger la séquence de dates comme nouvel index au dataframe
    df1.index = idx[:492]
    elNino['elnino{}'.format(elnin)]=df1
for x in elNino:
    df_daily = elNino[x].asfreq('D')
    df_daily['El Nino'] = df_daily['El Nino'].ffill()
    elNino[x]=df_daily
    elNino[x].reset_index(inplace=True)   
    elNino[x].rename(columns={'index':'start_date'},inplace=True)
    elNino[x]=elNino[x].loc[elNino[x]['start_date'] <= '2018-05-23 00:00:00']
    
merged_df_elnino=elNino['elnino1']
i=0
for x in elNino:
    if i!=0:
        print(x)
        merged_df_elnino = pd.merge(merged_df_elnino, elNino[x][['start_date', 'El Nino']], on='start_date', how='left')
    i+=1

merged_df_elnino['El Nino']=merged_df_elnino['El Nino'].str.strip().astype(float)
merged_df_elnino['El Nino_x']=merged_df_elnino['El Nino_x'].str.strip().astype(float)
merged_df_elnino['El Nino_y']=merged_df_elnino['El Nino_y'].str.strip().astype(float)
merged_df_elnino['start_date'] = merged_df_elnino['start_date'].drop_duplicates()
print(merged_df_elnino['start_date'].drop_duplicates())
merged_df_elnino['start_date'] = merged_df_elnino['start_date'].view('int64').astype(int) 

print(merged_df_elnino)

#Saves El Nino tensor

extract_time = merged_df_elnino['start_date'].drop_duplicates().sort_values()
extract_time =np.array(extract_time).tolist()
extract_features = np.array(merged_df_elnino.columns).tolist()
# Create the coordinate map
coordinate_map = {}
for time_idx, time in enumerate(extract_time):
    for features_idx, feature in enumerate(extract_features):
        coordinate_map[(time, feature)] = (time_idx, features_idx)

# Create an empty tensor with the desired dimensions
tensor_el_nino = th.empty((len(extract_time), len(extract_features)))

# Iterate over the dataframe rows and fill the tensor using the coordinate map
for columns in merged_df_elnino.columns:
    print(columns)
    tic()
    for _, row in merged_df_elnino.iterrows():
        time, feature = row['start_date'], row[columns]
        tensor_el_nino[coordinate_map[(time, columns)]] = feature
    toc()
# Print the filled tensor
print(tensor_el_nino.shape)

cache_dir = os.path.join('results', 'matrix')
# e.g., cache_dir = 'results/regression/shared/contest_precip_34w'

# if cache_dir doesn't exist, create it
if not os.path.isdir(cache_dir):
    os.makedirs(cache_dir)

# Filenames for data file to be stored in cache_dir
data_file = os.path.join(
    cache_dir, "El_NINO_TEST.tensor")

print("Saving multiarrays features to " + data_file)
th.save(tensor_el_nino, data_file)

print("Finished generating data matrix.")

elnino34
elnino4
0       1979-01-01
1       1979-01-02
2       1979-01-03
3       1979-01-04
4       1979-01-05
           ...    
14383   2018-05-19
14384   2018-05-20
14385   2018-05-21
14386   2018-05-22
14387   2018-05-23
Name: start_date, Length: 14388, dtype: datetime64[ns]
                start_date  El Nino_x  El Nino_y  El Nino
0       283996800000000000      24.72      26.41    28.36
1       284083200000000000      24.72      26.41    28.36
2       284169600000000000      24.72      26.41    28.36
3       284256000000000000      24.72      26.41    28.36
4       284342400000000000      24.72      26.41    28.36
...                    ...        ...        ...      ...
14383  1526688000000000000      23.67      27.73    29.07
14384  1526774400000000000      23.67      27.73    29.07
14385  1526860800000000000      23.67      27.73    29.07
14386  1526947200000000000      23.67      27.73    29.07
14387  1527033600000000000      23.67      27.73    29.07

[14388 rows x 4 column

## Creation of a time tensor

In [35]:
extract_time = filtered_df['start_date'].drop_duplicates().sort_values()
extract_time = th.tensor(extract_time)

# Name of cache directory for storing non-submission-date specific
# intermediate files
cache_dir = os.path.join('results', 'matrix')
# e.g., cache_dir = 'results/regression/shared/contest_precip_34w'

# if cache_dir doesn't exist, create it
if not os.path.isdir(cache_dir):
    os.makedirs(cache_dir)

# Filenames for data file to be stored in cache_dir
data_file = os.path.join(
    cache_dir, "time_TEST.tensor")

print("Saving multiarrays features to " + data_file)
th.save(extract_time, data_file)

print("Finished generating data matrix.")


Saving multiarrays features to results/matrix/time_TEST.tensor
Finished generating data matrix.


# Creation and saving of the final tensor
this step is time consuming (about 10 minutes per column), run it during the night.

In [ ]:
tic()
extract_latitudes = filtered_df['lat'].drop_duplicates().sort_values()
extract_longitudes = filtered_df['lon'].drop_duplicates().sort_values()
extract_time = filtered_df['start_date'].drop_duplicates().sort_values()
extract_latitudes =np.array(extract_latitudes).tolist()
extract_longitudes =np.array(extract_longitudes).tolist()
extract_time =np.array(extract_time).tolist()
extract_features = np.array(filtered_df.columns).tolist()
# Create the coordinate map
coordinate_map = {}
for lat_idx, lat in enumerate(extract_latitudes):
    print(lat_idx)
    for lon_idx, lon in enumerate(extract_longitudes):
        for time_idx, time in enumerate(extract_time):
            for features_idx, feature in enumerate(extract_features):
                coordinate_map[(lat, lon, time, feature)] = (lat_idx, lon_idx, time_idx, features_idx)

toc()

#to clear space on the CPU
del lat_lon_date_data
del date_data

# Create an empty tensor with the desired dimensions
tensor_3d = th.empty((len(extract_latitudes), len(extract_longitudes), len(extract_time), len(extract_features)))

# Iterate over the dataframe rows and fill the tensor using the coordinate map
for columns in filtered_df.columns:
    print(columns)
    tic()
    for _, row in filtered_df.iterrows():
        lat, lon, time, feature = row['lat'], row['lon'], row['start_date'], row[columns]
        tensor_3d[coordinate_map[(lat, lon, time, columns)]] = feature
    toc()
# Print the filled tensor
print(tensor_3d)




In the following box, we fill the NaN values using nearest neighbours values. If there is no data at one date, we copy and paste th data of the previous date

In [ ]:
from scipy.interpolate import griddata

tensor_filled = reshaped_tensor.clone()  # Create a copy of the tensor
# Iterate over each sample in the tensor
tic()
i=0
prev_sample_data = th.zeros(tensor_filled.shape[-3],tensor_filled.shape[-2],tensor_filled.shape[-1])
print(prev_sample_data.shape)
for sample in tensor_filled: #across the first dimension (temporal)
    i+=1
    if i%500 ==0: #to follow the code advance
        print(i)
        #break
    for channel in range(sample.shape[-1]): #across the last dimension (features)
        # Extract the channel
        channel_data = sample[:, :, channel]
        nan_count = th.isnan(channel_data).sum().item()
        #if nan_count !=0 :
            #print(nan_count)
        
        # Get the non-NaN coordinates
        non_nan_indices = np.where(np.isfinite(channel_data))
        non_nan_points = np.array(non_nan_indices).T

        # Get the NaN coordinates
        nan_indices = np.where(np.isnan(channel_data))
        nan_points = np.array(nan_indices).T

        
        #if it exists some non NaN values at this date and for this feature we can interpolate
        if len(non_nan_points) > 0: 
            # Interpolate the NaN values using the non-NaN values
            channel_data_nan_filled = griddata(
                non_nan_points,
                channel_data[non_nan_indices],
                nan_points,
                method='nearest'  # Choose the interpolation method 'linear', 'nearest', 'cubic'
            )
            ####we have huge holes in datas, linear and cubic doesn't fill the big holes, only nearest can do it, even if it is less precise
       
            channel_data_nan_filled = th.from_numpy(channel_data_nan_filled).float()
        
        
            # Replace the NaN values with the interpolated values
            channel_data[nan_indices] = channel_data_nan_filled
            if nan_count !=0 :
                nan_count3 = th.isnan(channel_data).sum().item()
                if nan_count3 !=0:
                    print("Number of NaN values:", nan_count3)
            
        ##otherwise we just copy the values of the last time step (another sample) 
        else :
            if prev_sample_data is not None:
                #print('1',channel_data.shape)
                #print('2',prev_sample_data.shape)
                inter2 = prev_sample_data[:,:,channel].clone()
                channel_data = inter2
                nan_count3 = th.isnan(channel_data).sum().item()
                if nan_count3 !=0:
                    print("Number of NaN values:", nan_count3)
                
        intermediate = channel_data.clone()
        prev_sample_data[:,:,channel] = intermediate
        
   
toc()
# The tensor 'tensor_filled' now contains the inpainted values

In [ ]:
from scipy import interpolate
tensor_filled2 = tensor_filled.clone()
# Convert tensor to numpy array
for i in range(3,len(tensor_filled[0,0,0,:])):
    if i != 90: #useless remove it
        
        print(i)
        array = tensor_filled2[:,:,:,i].numpy()
        print(array.shape)
####CAUTION : Have to change it and adapt to each column , or find the reason of why some datas are extremely close to 0 or giat. These thresholds only managee tmperature tm2mp problems
        #threshold1 = 1e-6  # Adjust the threshold as needed
        #threshold2 = 1e4 #careful, these thresholds aare valid for tmp2m but I didn't check elsewhere
    # Identify strange values and replace them with NaN
        #array[array < threshold1] = np.nan
        #array[array > threshold2] = np.nan

    # Get the indices of NaN values
        nan_indices = np.isnan(array)

    # Get the indices of non-NaN values
        non_nan_indices = np.argwhere(~nan_indices)

    # Create 1D arrays of indices for non-NaN values
        x_indices = non_nan_indices[:, 2]
        y_indices = non_nan_indices[:, 1]
        z_indices = non_nan_indices[:, 0]
        values = array[non_nan_indices[:, 0], non_nan_indices[:, 1], non_nan_indices[:, 2]]

    # Create a 3D interpolation function using nearest neighbor method
        interp_func = interpolate.NearestNDInterpolator((x_indices, y_indices, z_indices), values)

    # Get the indices of NaN values
        nan_indices = np.isnan(array)

    # Get the coordinates of NaN values
        nan_coordinates = np.argwhere(nan_indices)

    # Fill NaN values with the interpolated values from nearest neighbors
        for coord in nan_coordinates:
            x, y, z = coord
            array[x, y, z] = interp_func(z, y, x)  # Reversed coordinates due to numpy indexing

    # Convert the numpy array back to a tensor
        tensor_filled2[:,:,:,i] = th.from_numpy(array)
        print(tensor_filled2[:,:,:,i])
print(tensor_filled2)

nan_count = th.isnan(tensor_filled2).sum().item()
print("Number of NaN values:", nan_count)

Saving the tensor 

In [ ]:

# Name of cache directory for storing non-submission-date specific
# intermediate files
cache_dir = os.path.join('results', 'matrix')
# e.g., cache_dir = 'results/regression/shared/contest_precip_34w'

# if cache_dir doesn't exist, create it
if not os.path.isdir(cache_dir):
    os.makedirs(cache_dir)

# Filenames for data file to be stored in cache_dir
if gt_id == "contest_tmp2m" :
    if target_horizon == "56w" :
        data_file = os.path.join(cache_dir, "data_tmp2m56_new.tensor")
    if target_horizon == "34w" :
        data_file = os.path.join(cache_dir, "data_tmp2m34_new.tensor")
if gt_id == "contest_precip" :
    if target_horizon == "56w" :
        data_file = os.path.join(cache_dir, "data_precip56_new.tensor")
    if target_horizon == "34w" :
        data_file = os.path.join(cache_dir, "data_precip34_new.tensor")

print("Saving multiarrays features to " + data_file)
th.save(tensor_filled2, data_file)

print("Finished generating data matrix.")
